# Modern Portfolio Theory

- Modern Portfolio Theory (MPT) is a Nobel Prize-winning economic theory. 
- It explains how risk-averse investors can construct portfolios to optimize or maximize expected return based on a given level of market risk. 
- <a href = "https://en.wikipedia.org/wiki/Harry_Markowitz">Harry Markowitz</a> pioneered this theory in his paper <a href = "https://onlinelibrary.wiley.com/doi/full/10.1111/j.1540-6261.1952.tb01525.x">Portfolio Selection</a> , which was published in the Journal of Finance in 1952. He was later awarded a Nobel Prize for his work on modern portfolio theory.
- Modern Portfolio Theory suggests diversification of all your securities and asset classes and not putting all your eggs in one basket. It emphasises the importance of portfolios, diversification, risk and the connections among different kinds of securities.



In [2]:
pip install yfinance pandas numpy matplotlib requests beautifulsoup4 --break-system-packages

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
import yfinance as yf

nifty50 = yf.download("^NSEI", start="2019-01-01", end="2025-03-31")['Close']

[*********************100%***********************]  1 of 1 completed


In [9]:
nifty50 = nifty50.rename(columns = {'^NSEI': 'Nifty100'})
nifty50.head()

Ticker,Nifty100
Date,
2019-01-02,10792.500000
2019-01-03,10672.250000
2019-01-04,10727.349609
2019-01-07,10771.799805
2019-01-08,10802.150391


In [13]:
import requests
import pandas as pd

def fetch_mutual_fund_nav(api_url):
    """
    Fetches mutual fund NAV data from the given mfapi.in URL, filters between
    2019-01-01 and 2025-03-31, and returns a DataFrame with only 'date' and 'nav'.

    Parameters:
    - api_url (str): API endpoint of the mutual fund.

    Returns:
    - pd.DataFrame: Filtered DataFrame with 'date' and 'nav' columns.
    """
    try:
        response = requests.get(api_url)
        response.raise_for_status()
        data = response.json().get("data", [])

        # Load into DataFrame
        df = pd.DataFrame(data)[["date", "nav"]]

        # Convert and filter
        df["date"] = pd.to_datetime(df["date"], format="%d-%m-%Y")
        df["nav"] = df["nav"].astype(float)

        # Filter date range
        mask = (df["date"] >= "2019-01-01") & (df["date"] <= "2025-03-31")
        df = df.loc[mask]

        return df.sort_values("date").reset_index(drop=True)

    except Exception as e:
        print(f"Error: {e}")
        return pd.DataFrame(columns=["date", "nav"])

# Example usage
url = "https://api.mfapi.in/mf/122639"
df = fetch_mutual_fund_nav(url)


In [14]:
df.head()

,date,nav
0,2019-01-01,24.1411
1,2019-01-02,24.0361
2,2019-01-03,24.0002
3,2019-01-04,23.7828
4,2019-01-07,23.9844
